<br>
<br>
<div style="font-size:24px; font-weight:bold">ProxiPal PLA-Dx | Experiment Processing & Performance Report</div>
<br>
<div style="font-size:14px; font-weight:bold">Title and Release Version of Your PLA-Dx Test (Document v#.##), Ref: PxK ### </div>
<hr style="border: none; border-top: 3px solid black;">

__Scope:__ This notebook template provides a basic performance report for a single experiment and the tools required to add  test data to the central database "mastertable".  

__Features:__ While one report format is being maintained, flexibility is required to accomodate all users.  
- Use the _"Toggle Code Cells"_ button to improve readability  
- Check that /data /analysis folders contain necessary files  
- Advanced users can use code cells to modify the notebook or add functionality  
- Automatic sample estimation using either simple linear regression (default) or with multiple models  
- Use instrument Cq thresholds or LinReg  
- Match sample submission details or search for missing information  
- Batch process multiple experiments  
  
<div style="font-size:14px; font-weight:bold; color:red">Hit "Cell" > "Run All" to initialise the report</div>

#### Oustanding updates & bugfixes  
Variables need to be described in the experiment conditions  
Variables need to be shown on the report for each well  
Variables need to be described for the standards reporting  

In [ ]:
import ipywidgets as widgets
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, Javascript, clear_output, Image, Markdown


class CodeToggler:
    def __init__(self):
        self.is_visible = True  # Track state
        
        # Improved JavaScript with state tracking and all cells handling
        self.js_code = """
        var jupyterCodeToggler = {
            isVisible: true,
            
            toggleCodeCells: function() {
                // Get all input cells including those before the button
                var codeCells = document.querySelectorAll('div.input');
                var newDisplay = this.isVisible ? 'none' : 'block';
                
                // Update all cells
                codeCells.forEach(function(cell) {
                    cell.style.display = newDisplay;
                });
                
                // Toggle state
                this.isVisible = !this.isVisible;
                
                // Store state in localStorage for persistence
                localStorage.setItem('jupyterCodeTogglerState', this.isVisible);
            },
            
            initializeState: function() {
                // Restore state from localStorage or default to visible
                var savedState = localStorage.getItem('jupyterCodeTogglerState');
                this.isVisible = savedState === null ? true : savedState === 'true';
                
                // Apply initial state
                var codeCells = document.querySelectorAll('div.input');
                var display = this.isVisible ? 'block' : 'none';
                codeCells.forEach(function(cell) {
                    cell.style.display = display;
                });
            }
        };
        
        // Initialize state when the notebook loads
        jupyterCodeToggler.initializeState();
        """
        
        # Create and configure the button
        self.button = widgets.Button(
            description="Toggle Code Cells",
            layout=widgets.Layout(width='250px'),
            tooltip="Click to show/hide code cells"
        )
        
        # Define the toggle function
        def toggle_code_cells(button):
            display(Javascript("jupyterCodeToggler.toggleCodeCells();"))
            self.is_visible = not self.is_visible
            button.description = "Hide Code Cells" if self.is_visible else "Show Code Cells"
            
        # Bind the function to the button
        self.button.on_click(toggle_code_cells)
    
    def initialize(self):
        """Initialize the toggler in the notebook"""
        # First inject the JavaScript code
        display(HTML(f"<script>{self.js_code}</script>"))
        # Then display the button
        display(self.button)

# Create and initialize the toggler
toggler = CodeToggler()
toggler.initialize()

<div style="font-size:24px; font-weight:bold">Execution environment  </div>

In [ ]:
from ProxiPal import *

# Data Folder Functions  
The __/data__ folder is general access and where users should save their experiment files. Users can access csv outputs of all displayed tables from the relevant experiment's __/exports__ folder.

## Review available experimental data  
Use this table to confirm which experiments have the requisite files for an analysis.

In [ ]:
# Check for an eds > txt export
eds2txt_match_list, eds2txt_match_dict = find_matched_filenames(base_path, read_export = True)
# Check for a csv file with the same name as the eds file
eds2csv_match_list, eds2csv_match_dict = find_matched_filenames(base_path, native_format = '.eds', export_format = '.csv', read_export = True)
# Review matched filenames
df_pivot = review_matched_filenames(eds2txt_match_dict, eds2csv_match_dict)
# Create a df_pivot without the path_key column for display purposes
df_without_path_key = df_pivot.drop(columns="path_key")

def display_pandas_table_rev(button):
    clear_output()
    print("Data files reviewed on: ", datetime.now().strftime("%A %d/%m/%y %H:%M"))
    display(button_box_rev)
    pd.set_option('display.max_rows', None)
    display(df_without_path_key)

# Create button
button_pandas_rev = widgets.Button(description="Pandas Checklist", layout=widgets.Layout(width='250px'))

# Connect buttons to functions
button_pandas_rev.on_click(display_pandas_table_rev)

# Create a horizontal box with your buttons
button_box_rev = widgets.HBox([button_pandas_rev])

# Display the box
display(button_box_rev)


## Link Experiment Data and Instrument Parameters  
Experimental data from the user-submitted csv/xlsx files are matched to qPCR instrument parameters that are extracted from a .txt export file.
- Matching to instrument parameters is presently only supported for ABI Quant Studio software.
- Users of non-ABI instruments can still analyse data without instrument parameters being recorded (see ProxiPal manual).

In [ ]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

# Column names of interest
columns_of_interest = ['filepath_txt', 'Kit_#', 'InstrumentType', 'ExperimentRunStartTime',
                       'QuantificationCycleMethod', 'Stage/CyclewhereAnalysisisperformed',
                       'Chemistry', 'PassiveReference', 'BlockType', 'InstrumentSerialNumber']

# Create input text field
style = {'description_width': 'initial'}
input_text = widgets.IntText(description='Select experiment index to analyze:', value=9999, style=style)

# Create a button
button_params = widgets.Button(description='Link CSV & EDS data',
                               layout=widgets.Layout(width='250px'))

# Define your function
def pandas_link_params(path_key):
    # Assuming the function ProxiPal.create_data_metatable is already defined
    meta_list = create_data_metatable(eds2txt_match_dict, eds2csv_match_dict, path_key)

    # Calibration columns
    original_calibration_columns = ['CalibrationBackgroundisexpired', 'CalibrationPureDyeROXisexpired',
                                   'CalibrationPureDyeSYBRisexpired', 'CalibrationRNasePisexpired',
                                   'CalibrationROIisexpired', 'CalibrationUniformityisexpired']

    # Check for presence of calibration columns in meta_list[0]
    calibration_columns = []
    missing_columns = []
    for col in original_calibration_columns:
        if col in meta_list[0].columns:
            calibration_columns.append(col)
        else:
            missing_columns.append(col)

    # Print missing calibration columns
    if missing_columns:
        print("Calibration standard: ", ', '.join(missing_columns), " is not available.")

    # Initialize a new DataFrame
    new_df = pd.DataFrame(columns=['Parameter', 'Status'])

    # Populate the new DataFrame
    for col in columns_of_interest:
        if col in meta_list[0].columns:
            unique_values = meta_list[0][col].dropna().unique()  # Get unique values excluding NaNs
            unique_values_str = ', '.join(map(str, unique_values))  # Convert to string and join with commas
            new_row = pd.DataFrame({'Parameter': [col], 'Status': [unique_values_str]})
            new_df = pd.concat([new_df, new_row], ignore_index=True)
        else:
            print(f"Column '{col}' not found in the DataFrame.")

    # Calibration check
    calibration_status = []
    for col in calibration_columns:
        if (meta_list[0][col] != 'No').any():  # If any value in the column is not 'No'
            calibration_status.append(col.replace('Calibration', '').replace('isexpired', ''))
    if len(calibration_status) == 0:
        new_row = pd.DataFrame({'Parameter': ['Calibration Check'], 'Status': ['OK']})
        new_df = pd.concat([new_df, new_row], ignore_index=True)
    else:
        status_str = ', '.join(calibration_status)
        new_row = pd.DataFrame({'Parameter': ['Calibration Check'],
                                'Status': ['Calibration expired: ' + status_str]})
        new_df = pd.concat([new_df, new_row], ignore_index=True)

    display(new_df.style.set_properties(subset=['Status'], **{'width': '650px'}))


# Define what happens when the button is clicked
def button_link_params(b):
    # Disable the button
    button_params.disabled = True
    # Fetch the path_key from df_pivot based on input_text value
    path_key = df_pivot.at[input_text.value, 'path_key']
    pandas_link_params(path_key)
    print("CSV & EDS Link Complete:", datetime.now().strftime("%A %d/%m/%y %H:%M"))

# Set the on_button_clicked function to be called when the button is clicked
button_params.on_click(button_link_params)

# Create a horizontal box with the text field and the button
hbox = widgets.HBox([input_text, button_params])

# Display the horizontal box
display(hbox)


## Show Experiment Plan  
Experiment plans are extracted from the user-submitted csv/xlsx planning template.

In [ ]:
def show_plan(button):
    # Disable the button
    button_plan.disabled = True
    
    #     Use the input_text value from the previous cell
    input = input_text.value
    exports_path = Path(data_folder / df_pivot['experiment'][input] / df_pivot['analysis'][input] / "exports")
    metatable = pd.read_csv(exports_path / "metatable.csv")

    unique_values = metatable["experiment_plan"].unique().tolist()
    list_of_strings = [f"{i+1}. {value}" for i, value in enumerate(unique_values[0].split('\n'))]

    display(Markdown('\n'.join(list_of_strings)))

button_plan = widgets.Button(description="Show Experiment Plans", layout=widgets.Layout(width='250px'))
button_plan.on_click(show_plan)
display(button_plan)


In [ ]:
def show_plate(button):
    # Disable the button
    button_plate.disabled = True
    
    #     Use the input_text value from the previous cell
    input = input_text.value
    exports_path = Path(data_folder / df_pivot['experiment'][input] / df_pivot['analysis'][input] / "exports")
    metatable = pd.read_csv(exports_path / "metatable.csv")
    
    fig1 = create_plate_visualization(metatable, palette = None, font_size = 14, values_decimal_places=0)
    plt.close('all')
    plt.figure(fig1)
    plt.show()

button_plate = widgets.Button(description="Show Plate Plan", layout=widgets.Layout(width='250px'))
button_plate.on_click(show_plate)
display(button_plate)


In [ ]:
def show_plate2(button):
    # Disable the button
    button_plate.disabled = True
    
    #     Use the input_text value from the previous cell
    input = input_text.value
    exports_path = Path(data_folder / df_pivot['experiment'][input] / df_pivot['analysis'][input] / "exports")
    metatable = pd.read_csv(exports_path / "metatable.csv")
    
    fig1 = create_plate_visualization(metatable, palette = None, font_size = 8,
                                     value1=('usr_raw_ng/L', 'ng/L: '), value2=('usr_mean_ng/L', 'avg ng/L: '),
                                    heatmap = True, heatmap_palette = 'Spectral', values_decimal_places=0, heatmap_decimal_places=1)
    plt.close('all')
    plt.figure(fig1)
    plt.show()

button_plate2 = widgets.Button(description="Ct Value Heatmap", layout=widgets.Layout(width='250px'))
button_plate2.on_click(show_plate2)
display(button_plate2)


## Show Amplification and Melt Curve Plots  
Reaction plots are best viewed exploerd in the qPCR instrument software. Here we simply import saved plot images (saved as _"Amplification Plot.jpg"_ and _"Melt Curve Plot.jpg"_) for reporting purposes. 

In [ ]:
def display_jpegs(button):
    # Disable the button
    button_jpegs.disabled = True
    
    input = input_text.value
    experiment_folder = Path(data_folder / df_pivot['experiment'][input] / df_pivot['analysis'][input])
    amp_plot = experiment_folder / 'Amplification Plot.jpg'
    melt_plot = experiment_folder / 'Melt Curve Plot.jpg'
    
    if amp_plot.is_file():
        display(Image(filename=amp_plot))
    else:
        print("Amplification Plot.jpg not found!")

    if melt_plot.is_file():
        display(Image(filename=melt_plot))
    else:
        print("Melt Curve Plot.jpg not found!")
        
    print("Plots displayed on: ", datetime.now().strftime("%A %d/%m/%y %H:%M"))

# Create a button
button_jpegs = widgets.Button(description="Display qPCR Plots", layout=widgets.Layout(width='250px'))

# Assign the function to the button's on_click event
button_jpegs.on_click(display_jpegs)

# Display the button
display(button_jpegs)


## Calculate Concentrations  
This section applies python methods from the numpy, pandas, scikit-learn, plotly and rdmlpython packages. ProxiPal will check the experiment folder for any available .rdml exports and also process these using Ruijter's efficiency-corrected LinRegPCR approach.

Calculations do not modify user-submitted files, and all calculated tables are saved to the relevant __/exports__ folder.

### Quantitate with Simple Linear Regression  (Ct & rdml_Cq)
The NFL IC-PLA qPCR demonstrates excellent linearity and parallelism with respect to NfL concentration. For this reason qPCR standards of quantitation can be reliably applied. Our default quantitation model is the most conventional and processes experimental data using user-defined instrument "Ct" values and, if available, log2-transformed LinReg "N0" values. Neither of these cycling statistics feature PCR efficiency-correction.  

In [ ]:
# Create a second button
button_lin_reg = widgets.Button(description='Apply Linear Regression',
                         layout=widgets.Layout(width='250px'))
# Define the function to run when the second button is clicked
def apply_lin_reg(b):
    # Disable the button
    button_lin_reg.disabled = True
    
#   Use the input_text value from the previous cell
    input = input_text.value
    path = Path(data_folder / df_pivot['experiment'][input] / df_pivot['analysis'][input] / "exports" / "metatable.csv")
    metatable = pd.read_csv(path) 
    py_metatable = create_py_metatable(metatable, threshold_type='ct', rdml_check=True, export=True)

    print("Standards calculated: ", datetime.now().strftime("%A %d/%m/%y %H:%M"))
# Set the on_button2_clicked function to be called when the second button is clicked
button_lin_reg.on_click(apply_lin_reg)
# Display the second button
display(button_lin_reg)

### Conventional Ct Analyses

#### Display Standards  
Automated calculations are prefixed with "py_" and all standard curve assignments should be made by users in the csv/xlsx tableson a per well basis.  
- Users should assign consistent threshold value for all reactions when exporting Ct values; for ABI systems we use 0.1 with the SYBR assays.

In [ ]:
def process_metatable_py(metatable_path):
    from IPython.display import clear_output, display
    
    # Read the metatable directly
    py_metatable = pd.read_csv(metatable_path, low_memory=False)
    
    # Clear any existing output
    clear_output(wait=True)
    
    # Create and display the standards plot first
    print("\nStandards Plot:")
    plt.figure(figsize=(8, 6))  # Create figure explicitly
    
    result = plot_slr_standards(metatable=py_metatable, 
                              threshold_type='ct',
                              std0_status='exc_std0',
                              figsize=(8, 6),
                              separate_plots=False)

    plt.show()  # Force plot display
    
    # Print a separator and header for the tables section
    print("\n" + "="*50 + "\nStandards Tables:")
    
    # Get filepath from metatable
    filepath = py_metatable['filepath_csv'].unique().tolist()[0]
    
    # Get the experiment tables
    SLR_experiment_tables = extract_experiment_tables(
        df=py_metatable,
        filepath_csv=filepath,  # Use filepath from metatable
        quant_model='SLR',
        threshold_type='ct',
        transform_x='log10(x)',
        std0_status='exc_std0',
        sample_type = 'standards',
        simple_headers=True
    )
    
    # Display report tables for each standard
    for key in SLR_experiment_tables.keys():
        if 'report_table' in key:
            print(f"\n{key}:")
            display(SLR_experiment_tables[key])

def show_standards_py(event):
    # Disable the button
    button_show_standards.disabled = True
    
    # Construct the direct path to py_metatable.csv
    metatable_path = Path(data_folder / df_pivot['experiment'][input_text.value] / 
                         df_pivot['analysis'][input_text.value] / "exports" / "py_metatable.csv")
    
    # Process the metatable
    process_metatable_py(metatable_path)
    print("\nData displayed on: ", datetime.now().strftime("%A %d/%m/%y %H:%M"))

# Create a button
button_show_standards = widgets.Button(description="Display Standards", 
                        layout=widgets.Layout(width='250px'))    

# Attach the function to the button
button_show_standards.on_click(show_standards_py)

# Display the button
display(button_show_standards)

#### QC sample performance  
QC sample performance calculations are jupyter report only and not saved to the experiment data.  
Nomenclature for fail raw_ng/L and fail mean_ng/L is __well position : ng/L : no. of stdev from expected value__

#### Imprecision- Validation Set

In [ ]:
# Create the buttons
button_pandas_samples = widgets.Button(description="Pandas QC Table", layout=widgets.Layout(width='250px'))

# Specify one manually
qc_dict = {
    'sample_id': ['NFL-LoQ', 'NFL-QC-L[6]', 'NFL-QC-M[19]', 'NFL-QC-H[91]'],
    'mean': [2, 6, 19, 91],
    'stdev': [0.34, 0.81, 2.19, 9.91],
    'stdev_range=2': ['[1, 3]', '[4, 8]', '[15, 23]', '[71, 111]'],
    'fail raw_ng/L': [None, None, None, None],
    'fail mean_ng/L': [None, None, None, None],
}

# Create DataFrame
qc_df = pd.DataFrame(qc_dict)

# Define the data display function for Pandas QC Table
def compare_QC_validation_set(event):
    input_val = input_text.value

    clear_output()
    display(widgets.HBox([button_pandas_samples]))

    metatable_path = Path(
        data_folder
        / df_pivot['experiment'][input_text.value]
        / df_pivot['analysis'][input_text.value]
        / "exports"
        / "py_metatable.csv"
    )

    py_metatable = pd.read_csv(metatable_path, low_memory=False)
    updated_QC_df = process_qc_data(qc_df, py_metatable)

    pd.set_option('display.max_rows', None)
    display(updated_QC_df)

button_pandas_samples.on_click(compare_QC_validation_set)
display(widgets.HBox([button_pandas_samples]))


#### Imprecision- Custom Set
This section will recalculate the quality control tables specifying specfic experiments.  
This requires mastertable access. Refer to the "Available Experiments Checklist" and identify which experiments to use.  
You might use this feature to:  
- Refer to a set group of imprecision experiments performed during QC validation; this assumes instrument calibration and assay stability since validation
- Recalulate QC boundaries based on more recent experiments; this permits drift to be accomodated


In [ ]:
class ImprecisionAnalysisProcessor:
    def __init__(self):
        self.password = proxipal_password

    def run_analysis(self, button):
        button.disabled = True
        clear_output()
        display(widgets_box_imprecision)

        if self.password_input.value == self.password:
            mastertable, _ = load_most_recent_mastertable(base_path / 'exports')

            # Parse user input for selected rows
            try:
                selected_rows = [int(x.strip()) for x in self.row_input.value.split(',') if x.strip()]
            except ValueError:
                print("Invalid input for selected rows. Use comma-separated integers.")
                return

            df_pivot_list = [f"{k}.csv" for k in df_pivot.loc[selected_rows, 'path_key']]
            mastertable_slice = mastertable[mastertable['filepath_csv'].isin(df_pivot_list)]

            analyse_ss = analyze_sample_stats(
                data=mastertable_slice,
                model_type='SLR; log10(x); exc_std0; ct; raw_ng/L',
                decimal_places=self.decimal_dropdown.value,
                format_timing=self.format_timing_dropdown.value
            )

            qc_df = build_qc_df(
                analyse_ss,
                sample_id=['NFL-QC-L', 'NFL-QC-M', 'NFL-QC-H'],
                stdev_range=2
            )

            qc_results = process_qc_data(qc_df, mastertable_slice)
            display(qc_results)
        else:
            print("Incorrect password. Try again.")

imprecision_processor = ImprecisionAnalysisProcessor()

imprecision_processor.password_input = widgets.Password(
    description='Password', layout=widgets.Layout(width='200px')
)

imprecision_processor.row_input = widgets.Text(
    description='Rows', placeholder='e.g. 0,1,2', layout=widgets.Layout(width='300px')
)

imprecision_processor.decimal_dropdown = widgets.Dropdown(
    options=['integer', 1, 2],
    value=1,
    description='Decimals',
    layout=widgets.Layout(width='300px')
)

imprecision_processor.format_timing_dropdown = widgets.Dropdown(
    options=['before', 'after'],
    value='before',
    description='Timing',
    layout=widgets.Layout(width='250px')
)

button_analyse_imprecision = widgets.Button(
    description="Run Imprecision Analysis", layout=widgets.Layout(width='250px')
)

button_analyse_imprecision.on_click(imprecision_processor.run_analysis)

widgets_box_imprecision = widgets.HBox([
    imprecision_processor.password_input,
    imprecision_processor.row_input,
    imprecision_processor.decimal_dropdown,
    imprecision_processor.format_timing_dropdown,
    button_analyse_imprecision
])

info_text = widgets.HTML(
    "<span style='font-size:13px;'>"
    "Select row indexes, decimal format, and calculation timing.<br>"
    "‘Decimals’ controls rounding precision; ‘Timing’ controls when rounding is applied."
    "</span>"
)

info_text = widgets.HTML(
    "<span style='font-size:13px;'>"
    "Row indexes: Choose from 'Available Experiments Checklist' Table <br>"
    "Timing: Assay acceptance is based on the QC range mean +/- 2*stdev. <br>"
    "'before' applies rounding prior to 2*stdev is calculated (default) <br>"
    "'after' applies rounding prior to 2*stdev is calculated <br>"
    "When rounding applies can greatly affect the acceptance limits"
    "</span>"
)

display(widgets.VBox([widgets_box_imprecision, info_text]))



#### Display Samples  
Displays the sample report table generated during concentration calculation.

In [ ]:
# Create the buttons
button_pandas_samples = widgets.Button(description="Pandas Sample Table", layout=widgets.Layout(width='250px'))

# Define the data display function for Pandas Sample Table
def display_ct_samples(event):
    input_val = input_text.value
    
    # Clear previous output first
    clear_output()
    # Show buttons immediately
    display(widgets.HBox([button_pandas_samples]))
    
    metatable_path = Path(data_folder / df_pivot['experiment'][input_text.value] / 
                         df_pivot['analysis'][input_text.value] / "exports" / "py_metatable.csv")
    # Read the metatable directly and process it
    py_metatable = pd.read_csv(metatable_path, low_memory=False)
    # Get filepath from metatable
    filepath = py_metatable['filepath_csv'].unique().tolist()[0]
    
    result = extract_experiment_tables(df = py_metatable, 
                                     filepath_csv = filepath, 
                                     quant_model = 'SLR', 
                                     threshold_type = 'ct', 
                                     transform_x='log10(x)', 
                                     std0_status='exc_std0', 
                                     sample_type = 'samples', 
                                     simple_headers=True)
    
    pd.set_option('display.max_rows', None)
    display(result['sample_report_table'])

# Assign the event handlers to the buttons
button_pandas_samples.on_click(display_ct_samples)

# Display the buttons
display(widgets.HBox([button_pandas_samples]))

#### Display Well Table  
Displays the a filtered version of the py_metatable table generated during concentration calculation.

In [ ]:
# Create the buttons
button_pandas_wells = widgets.Button(description="Pandas Well Table", layout=widgets.Layout(width='250px'))

# Define the data display function for Pandas Sample Table
def display_ct_wells(event):
    input_val = input_text.value
    
    # Clear previous output first
    clear_output()
    # Show buttons immediately
    display(widgets.HBox([button_pandas_wells]))
    
    metatable_path = Path(data_folder / df_pivot['experiment'][input_text.value] / 
                         df_pivot['analysis'][input_text.value] / "exports" / "py_metatable.csv")
    # Read the metatable directly and process it
    py_metatable = pd.read_csv(metatable_path, low_memory=False)
    # Get filepath from metatable
    filepath = py_metatable['filepath_csv'].unique().tolist()[0]
    
    result = extract_experiment_tables(df = py_metatable, 
                                     filepath_csv = filepath, 
                                     quant_model = 'SLR', 
                                     threshold_type = 'ct', 
                                     transform_x='log10(x)', 
                                     std0_status='exc_std0', 
                                     sample_type = 'wells', 
                                     simple_headers=True)
    
    pd.set_option('display.max_rows', None)
    display(result['wells_table'])

# Assign the event handlers to the buttons
button_pandas_wells.on_click(display_ct_wells)

# Display the buttons
display(widgets.HBox([button_pandas_wells]))

### Amplication Efficiency Corrected Analyses Using LinReg/RDML  
LinReg is used to adjust threshold values for variations in cycling efficiency among individual reactions. The implementation we use here is N0; however we do not draw simple linear regression between linear concentration and linear N0; instead we use log10(concentration) vs -1.log2(N0) which we find leads to slightly better standard curve fits.  

#### Display Standard Curves  

In [ ]:
# Create a button
button_show_RDMLstandards = widgets.Button(description="Display rdml_log2N0 Standards", 
                        layout=widgets.Layout(width='250px'))

def process_metatable_rdml(metatable_path):
    from IPython.display import clear_output, display
    
    # Read the metatable directly
    py_metatable = pd.read_csv(metatable_path, low_memory=False)
    
    # Clear any existing output
    clear_output(wait=True)
    
    # Create and display the standards plot first
    print("\nStandards Plot:")
    plt.figure(figsize=(8, 6))  # Create figure explicitly
    result = plot_slr_standards(metatable=py_metatable, 
                              threshold_type='rdml_log2N0 (mean eff) - no plateau - stat efficiency',
                              std0_status='exc_std0',
                              figsize=(8, 6),
                              separate_plots=False)
    plt.show()  # Force plot display
    
    # Print a separator and header for the tables section
    print("\n" + "="*50 + "\nStandards Tables:")
    
    # Get filepath from metatable
    filepath = py_metatable['filepath_csv'].unique().tolist()[0]
    
    # Get the experiment tables
    SLR_experiment_tables = extract_experiment_tables(
        df=py_metatable,
        filepath_csv=filepath,  # Use filepath from metatable
        quant_model='SLR',
        threshold_type='rdml_log2N0 (mean eff) - no plateau - stat efficiency',
        transform_x='log10(x)',
        std0_status='exc_std0',
        sample_type = 'standards',
        simple_headers=True
    )
    
    # Display report tables for each standard
    for key in SLR_experiment_tables.keys():
        if 'report_table' in key:
            print(f"\n{key}:")
            display(SLR_experiment_tables[key])

def show_standards_py(event):
    # Disable the button
    button_show_RDMLstandards.disabled = True
    
    # Construct the direct path to py_metatable.csv
    metatable_path = Path(data_folder / df_pivot['experiment'][input_text.value] / 
                         df_pivot['analysis'][input_text.value] / "exports" / "py_metatable.csv")
    
    # Process the metatable
    process_metatable_rdml(metatable_path)
    print("\nData displayed on: ", datetime.now().strftime("%A %d/%m/%y %H:%M"))

# Attach the function to the button
button_show_RDMLstandards.on_click(show_standards_py)

# Display the button
display(button_show_RDMLstandards)

#### Display Samples

In [ ]:
# Create the buttons
button_pandas_RDMLsamples = widgets.Button(description="Pandas rdml_log2N0 Samples", layout=widgets.Layout(width='250px'))

# Define the data display function for Pandas Sample Table
def display_rdml_samples(event):
    input_val = input_text.value
    
    # Clear previous output first
    clear_output()
    # Show buttons immediately
    display(widgets.HBox([button_pandas_RDMLsamples]))
    
    metatable_path = Path(data_folder / df_pivot['experiment'][input_text.value] / 
                         df_pivot['analysis'][input_text.value] / "exports" / "py_metatable.csv")
    # Read the metatable directly and process it
    py_metatable = pd.read_csv(metatable_path, low_memory=False)
    # Get filepath from metatable
    filepath = py_metatable['filepath_csv'].unique().tolist()[0]
    
    result = extract_experiment_tables(df = py_metatable, 
                                     filepath_csv = filepath, 
                                     quant_model = 'SLR', 
                                     threshold_type = 'rdml_log2N0 (mean eff) - no plateau - stat efficiency', 
                                     transform_x='log10(x)', 
                                     std0_status='exc_std0', 
                                     sample_type = 'samples', 
                                     simple_headers=True)
    
    pd.set_option('display.max_rows', None)
    display(result['sample_report_table'])

# Assign the event handlers to the buttons
button_pandas_RDMLsamples.on_click(display_rdml_samples)

# Display the buttons
display(widgets.HBox([button_pandas_RDMLsamples]))

#### Display Wells

In [ ]:
# Create the buttons
button_pandas_RDMLwells = widgets.Button(description="Pandas rdml_log2N0 Wells", layout=widgets.Layout(width='250px'))

# Define the data display function for Pandas Sample Table
def display_wells_samples(event):
    input_val = input_text.value
    
    # Clear previous output first
    clear_output()
    # Show buttons immediately
    display(widgets.HBox([button_pandas_RDMLwells]))
    
    metatable_path = Path(data_folder / df_pivot['experiment'][input_text.value] / 
                         df_pivot['analysis'][input_text.value] / "exports" / "py_metatable.csv")
    # Read the metatable directly and process it
    py_metatable = pd.read_csv(metatable_path, low_memory=False)
    # Get filepath from metatable
    filepath = py_metatable['filepath_csv'].unique().tolist()[0]
    
    result = extract_experiment_tables(df = py_metatable, 
                                     filepath_csv = filepath, 
                                     quant_model = 'SLR', 
                                     threshold_type = 'rdml_log2N0 (mean eff) - no plateau - stat efficiency', 
                                     transform_x='log10(x)', 
                                     std0_status='exc_std0', 
                                     sample_type = 'wells', 
                                     simple_headers=True)
    
    pd.set_option('display.max_rows', None)
    display(result['wells_table'])

# Assign the event handlers to the buttons
button_pandas_RDMLwells.on_click(display_wells_samples)

# Display the buttons
display(widgets.HBox([button_pandas_RDMLwells]))

# Advanced Processing  
The current standard report lets users compare the performance of simple linear regressions performed with user-set thresholds in MS excel, python, and automated threshold selection with efficiency correction using linreg rdml. In advanced processing we test 3 quantitative models; simple linear regression (qPCR convention), 4-parameter logistic and 5 parameter logistic (immunoassay convention). All three models are fitted to all threshold types, with or without background subtraction, including "Ct", "rdml_Cq", "rdml_log2N0" and "rdml_N0" using both mean-corrected and individually-corrected efficiencies.  The result is produces concentration calculations and statistical reporting for 96 different models and combinations. This is quite computationally intensive and substantially increases the size of each experiment's *py_metatable.csv*.  
  
The advantage of doing this is to 1) test imprecision performance of all models, and 2) to identify the best fit for new targets in PLA development.

In [ ]:
# Create a second button
button_calc_all_models = widgets.Button(description='Calculate All Models',
                         layout=widgets.Layout(width='250px'))
# Define the function to run when the second button is clicked
def apply_all_models(b):
    # Disable the button
    button_calc_all_models.disabled = True
    
#   Use the input_text value from the previous cell
    input = input_text.value
    path = Path(data_folder / df_pivot['experiment'][input] / df_pivot['analysis'][input] / "exports" / "metatable.csv")
    metatable = pd.read_csv(path)
    py_metatable = calc_py_metatable_all_models(metatable, rdml_check=True, export=True)

# Set the on_button2_clicked function to be called when the second button is clicked
button_calc_all_models.on_click(apply_all_models)
# Display the second button
display(button_calc_all_models)

# Protected Queries: Current Experiment
These queries require use of the Master Table, a database built from the __/Data__, __/Samples__, and __/Quality folders__. For privacy reasons __/Samples__ and __/Quality__ information is password protected (default = _admin_).  

## Link Sample Submissions    
Matches sample submission information for the current experiment report, (_i.e. specimen type, collection type, medical abbreviation, age, gender_) from the __/Samples__ folder with the experiment data selected in step 1.2  
* Does not generate new megatables or mastertables  
* Sample-linked data is saved to __/user_downloads__ to accommodate folder permissions (if put in __/data__ sample info. would be leaked).  

In [ ]:
class Link_Sample_Processor:
    def __init__(self):
        self.master_table_filtered_grouped = None
        self.password = proxipal_password

    def load_and_filter_master(self, button):
        
        
        if self.password_input.value != self.password:
            print('Incorrect password. Try again.')
            return
        clear_output()
        display(button_box_link)
        
        master_table = load_most_recent_mastertable(base_path / 'exports')[0]
        
        mean_ng_col = 'SLR; log10(x); exc_std0; ct; mean_ng/L'
        
        # define columns to round
        numeric_columns = [mean_ng_col, 'age']
        # Process numeric columns
        for col in numeric_columns:
            master_table[col] = pd.to_numeric(master_table[col], errors='coerce')\
                .round(0)\
                .fillna(-999)\
                .astype(int)\
                .replace(-999, 'na')
        
        path_query = df_pivot['path_key'][input_text.value] + '.csv'
        # filter for select columns
#         master_table_filtered = master_table[master_table['filepath_csv']==path_query][['sample_id', 'tube_id', 'target', 'py_mean_ng/L', 'med_abbrev', 'age', 'sex', 'specimen_type', 'collection_type']].fillna('na')
        master_table_filtered = master_table[master_table['filepath_csv']==path_query][['sample_id', 'tube_id', 'target', 
                                                                                         mean_ng_col,
                                                                                        'med_abbrev', 'age', 'sex', 'specimen_type', 'collection_type', 'comments (anyone can use)']].fillna('na')
        
        # filter out usr_ignore == 1
        master_table_filtered = master_table_filtered[master_table['usr_ignore'] != 1]
        # filter out regular expressions used for standards
        master_table_filtered = master_table_filtered[~master_table['sample_id'].str.contains(r'\[(.*?)\]_', regex=True, na=False)]
        # rename mean_ng/L column for simple header
        master_table_filtered = master_table_filtered.rename(columns={mean_ng_col: 'mean_ng/L'})

        def custom_collapse(s):
            if s.nunique() == 1:
                return s.iloc[0]
            else:
                return '/'.join(s.astype(str).unique())

        self.master_table_filtered_grouped = master_table_filtered.groupby(['sample_id', 'tube_id']).agg(custom_collapse).reset_index()
        
        global recent_master_table_string        
        recent_master_table_string = str(load_most_recent_mastertable(base_path / 'exports')[1])
        print("ProxiPal master table retrieved: " + recent_master_table_string)
        print("""Default mean_ng/L = 'SLR; log10(x); exc_std0; ct; mean_ng/L'
        To change mean_ng/L model, change the Jupyter code cell directly")""")
    
    def display_pandas_link(self, button):
        # Clear previous output
        clear_output()
        print("ProxiPal master table retrieved: " + recent_master_table_string)
        
        # Disable the button
        if self.master_table_filtered_grouped is None:
            print('Please load and filter the master table first.')
            return

        # Clear previous output and display pandas table
        pd.set_option('display.max_rows', None)
        display(button_box_link)
        display(self.master_table_filtered_grouped)

    def export_link(self, button):
        if self.master_table_filtered_grouped is None:
            print('Please load and filter the master table first.')
            return

        # Write the header
        export_name = user_downloads / 'SampleLinkedData.csv'
        with open(export_name, 'w') as f:
            f.write(df_pivot['path_key'][input_text.value] + '   ' + datetime.now().strftime("%A %d/%m/%y %H:%M") + '\n')
        self.master_table_filtered_grouped.to_csv(export_name, mode='a', index=False)  # mode='a' means append
        print('CSV table exported to: ' + str(export_name))

Link_processor = Link_Sample_Processor()

# Password input
Link_processor.password_input = widgets.Password(description='Password')

# Original buttons
button_pandas_link = widgets.Button(description="Pandas Link Table", layout=widgets.Layout(width='250px'))
button_pandas_link.on_click(Link_processor.display_pandas_link)

button_export_link = widgets.Button(description="Export to CSV", layout=widgets.Layout(width='250px'))
button_export_link.on_click(Link_processor.export_link)

button_load_master = widgets.Button(description="Filter Master Table", layout=widgets.Layout(width='250px'))
button_load_master.on_click(Link_processor.load_and_filter_master)

# Create a horizontal box with your buttons
button_box_link = widgets.HBox([Link_processor.password_input, button_load_master, button_pandas_link, button_export_link])

# Display the box
display(button_box_link)


## Link Quality Control Information  
Quality monitoring currently uses a different system. It may be integrated here at a later date. 

# Protected Queries: Global Query  

## Batch All Experiments
ProxiPal processes every available experiments in the /data folder and merges experimental data into a single .csv table that is exported to the /data/exports folder. _NB:This function will overwrite all previous ProxiPal reports._ 
* Every experiment in /data will be processed. This can take up to 10 minutes.  
* Selecting the __Advanced Processing option__ will lead to a processing time of ~10mins _per experiment_
* A _data_megatable.csv_ will be created.  


In [ ]:
import ipywidgets as widgets
from IPython.display import clear_output, display

class BatchProcessor:
    def __init__(self):
        self.password = proxipal_password

    def batch_process_all(self, button):
        # Disable the button to prevent multiple clicks
        button.disabled = True
        clear_output()
        display(widgets_box_batch)

        password_input = self.password_input.value
        slice_input = self.slice_input.value.strip()
        calc_all_models_val = self.calc_all_models_dropdown.value  # Get dropdown value

        if password_input == self.password:
            df_pivot_slice = slice_input if slice_input else None

            # Call batch_py_metatables with user-defined calc_all_models flag
            batch_py_metatables(
                eds2txt_match_dict,
                eds2csv_match_dict,
                df_pivot_slice=df_pivot_slice,
                calc_all_models=calc_all_models_val  # ← Pass the dropdown value
            )

            # create data_megatable
            data_megatable = create_data_megatable(data_folder)
            print("New data_megatable.csv created: ", str(data_folder) + '/exports')
        else:
            print('Incorrect password. Try again.')

# Create an instance of the BatchProcessor class
batch_processor = BatchProcessor()

# Password widget
batch_processor.password_input = widgets.Password(
    description='Password',
    layout=widgets.Layout(width='275px')
)

# Slice input widget
batch_processor.slice_input = widgets.Text(
    description='Slice:',
    placeholder='e.g., 15:18 or 15:',
    layout=widgets.Layout(width='275px')
)

# Dropdown widget for calc_all_models
batch_processor.calc_all_models_dropdown = widgets.Dropdown(
    options=[True, False],
    value=False,
    description='Run Models*:',
    layout=widgets.Layout(width='275px')
)

# Button widget
button_batch_processor = widgets.Button(
    description='Batch',
    layout=widgets.Layout(width='250px')
)

# Link button to processing function
button_batch_processor.on_click(batch_processor.batch_process_all)

# Input layout
input_box = widgets.VBox([
    batch_processor.password_input,
    batch_processor.slice_input,
    batch_processor.calc_all_models_dropdown
])

# Final widget box layout
widgets_box_batch = widgets.HBox([input_box, button_batch_processor])

# Display
display(widgets_box_batch)

display(widgets.HTML(
    "<span style='font-size:13px;'>"
    "*Will test fit, and statistically assess, all available models (Linear, 4PL, 5PL, Exponential) "
    "against all available estimates of template (ct, N0 etc.). This can lead to processing times "
    "of ~10 min per experiment and csv table &gt;6000 columns."
    "</span>"
))


## Identify Missing Sample Information   
Creates a new master table from current __/Data__ and __/Samples__ folders. Searches the master table for instances of "orphan" entries:  
__"data(+) submission(-)":__ We have data, but no submission info.  
__"data(-) submission(+)":__ We have submission info, but no data.  
  
To ensure the most recent data is used.  
* A new _data_megatable.csv_ will be created.  
* A new _samples_megatable.csv_ will be created.  
* A new _'MissingSampleInfo.csv'_ is exportable on request.  

In [ ]:
class Orphan_Processor:
    def __init__(self):
        self.master_dict = None
        self.password = proxipal_password

    def create_new_master_table(self, button):
        password_input = self.password_input.value
        if password_input == self.password:
            clear_output()
            display(widgets_box_orphan)
            
            # create data_megatable
            data_megatable = create_data_megatable(data_folder)

            # create samples_megatable
            samples_megatable = create_samples_megatable(samples_folder, export=True)

            # create_master_table
            self.master_dict = create_master_table(match_type='TS')

            print("New Master Table created with Data and Sample Info: ", datetime.now().strftime("%A %d/%m/%y %H:%M"))
        
        else:
            print('Incorrect password. Try again.')            
            

    def pandas_orphan(self, button):
        clear_output()
        display(widgets_box_orphan)
        display(self.master_dict['master_df_orphan'])
    
    def csv_orphan(self, button):
        export_name = user_downloads / 'MissingSampleInfo.csv'
        with open(export_name, 'w') as f:
            f.write('Missing Sample Info' + '   ' + datetime.now().strftime("%A %d/%m/%y %H:%M") + '\n')

        self.master_dict['master_df_orphan'].to_csv(export_name, mode='a', index=False)  # mode='a' means append
        print('CSV table exported to: ' + str(export_name))

orphan_processor = Orphan_Processor()

# Password input
orphan_processor.password_input = widgets.Password(description='Password')

# Create buttons
button_pandas_orphan = widgets.Button(description="Pandas Orphan Table", layout=widgets.Layout(width='250px'))
button_export_orphan = widgets.Button(description="Export to CSV", layout=widgets.Layout(width='250px'))
button_create_master = widgets.Button(description="Create Master Table", layout=widgets.Layout(width='250px'))

# Connect buttons to functions
# button_qgrid_orphan.on_click(orphan_processor.qgrid_orphan)
button_pandas_orphan.on_click(orphan_processor.pandas_orphan)
button_export_orphan.on_click(orphan_processor.csv_orphan)
# button_create_master.on_click(Orphan_Processor.create_new_master_table)
button_create_master.on_click(orphan_processor.create_new_master_table)

# Create a horizontal box with your buttons
widgets_box_orphan = widgets.HBox([orphan_processor.password_input, button_create_master, button_pandas_orphan, button_export_orphan])

# Display the box
display(widgets_box_orphan)


## Create Master Table  
Collates all ProxiPal reports and sample submissions. Cross-checks all submitted sample IDs against all experimental data and matches all entries against a 1 row per reaction "master table" that contains all available data.  
* A _data_megatable.csv_ will be created  
* A _samples_megatable.csv_ will be created  
* A _mastertable.csv_ will be created  
* An _orphans table_ will be displayed

In [ ]:
class MasterTable_Processor:
    def __init__(self):
        self.password = proxipal_password

    def batch_process_all(self, button):
        # Disable the button
        button.disabled = True

        clear_output()
        display(widgets_box_master)

        password_input = self.password_input.value
        if password_input == self.password:
            # create data_megatable
            data_megatable = create_data_megatable(data_folder)
            print("New data_megatable.csv created: ", str(data_folder) + '/exports')

            # create samples_megatable
            samples_megatable = create_samples_megatable(samples_folder, export=True)
            print("New samples_megatable.csv created: ", str(samples_folder) + '/exports')

            # create_master_table
            master_dict = create_master_table(match_type='TS')

            print("New mastertable.csv created: ", str(base_path) + '/exports')
            print("Samples without Data or Submission info are shown in the table below")
                    
        else:
            print('Incorrect password. Try again.')

# Create an instance of the MasterTable_Processor class
master_processor = MasterTable_Processor()

# Create password widget
master_processor.password_input = widgets.Password(description='Password', layout=widgets.Layout(width='275px'))

# Create button widget
button_master_processor = widgets.Button(description='Create Master', layout=widgets.Layout(width='250px'))

# Execute function when button is clicked
button_master_processor.on_click(master_processor.batch_process_all)

# Create a horizontal box with your widgets
widgets_box_master = widgets.HBox([master_processor.password_input, button_master_processor])

# Display the box
display(widgets_box_master)


## Add Raw Instrument Data to Master Table  
Will extract all raw data from every quantstudio exported .txt file and merge the data for every individual reaction to the most recent master table in the database. Raw instrument data includes 'Raw Data', 'Amplification Data', 'Multicomponent Data', and 'Melt Curve Raw Data'.  
* An existing _mastertable.csv_ is loaded  
* A _mastertable_wInstrumentData.csv_ is created

In [ ]:
class Batch_Instr_Processor:
    def __init__(self):
        self.password = proxipal_password

    def batch_process_all(self, button):
        # Disable the button
        button.disabled = True

        clear_output()
        display(widgets_box_batch_instr)

        password_input = self.password_input.value
        if password_input == self.password:
            # Check for an eds > txt export
            eds2txt_match_list, eds2txt_match_dict = find_matched_filenames(data_folder, read_export = True)
            
            # Check for a csv file with the same name as the eds file
            eds2csv_match_list, eds2csv_match_dict = find_matched_filenames(data_folder, native_format = '.eds', export_format = '.csv', read_export = True)
            
            # Review matched filenames
            df_pivot = review_matched_filenames(eds2txt_match_dict, eds2csv_match_dict)
            
            # Extract raw values from all .eds exports into one table
            master_instr_df = build_master_instr_df(df_pivot, data_folder)
            
            # Load mastertable
            mastertable, mastertable_file = load_most_recent_mastertable(base_path / 'exports')
            
            # Merge with the master table
            mastertable_expanded = pd.merge(mastertable, master_instr_df, how='inner', on=['well', 'filepath_txt'])
                
            curr_time = datetime.now().strftime("%Y%m%d %H-%M")[2:].replace(' ', '_T')
            
            filename = curr_time + ' mastertable_wInstrumentData.csv'     
                                            
            # Save the table to /python folder for future use.
            mastertable_expanded.to_csv(Path(base_path / 'exports' / filename))

            print("Raw Instrument Data added to Master Table:", filename, ' ', datetime.now().strftime("%A %d/%m/%y %H:%M"))


        else:
            print('Incorrect password. Try again.')

# Create an instance of the BatchProcessor class
batch_instr_processor = Batch_Instr_Processor()

# Create password widget
batch_instr_processor.password_input = widgets.Password(description='Password', layout=widgets.Layout(width='275px'))

# Create button widget
button_batch_instr_processor = widgets.Button(description='Add All Instrument Data', layout=widgets.Layout(width='250px'))

# Execute function when button is clicked
button_batch_instr_processor.on_click(batch_instr_processor.batch_process_all)

# Create a horizontal box with your widgets
widgets_box_batch_instr = widgets.HBox([batch_instr_processor.password_input, button_batch_instr_processor])

# # Display the box
display(widgets_box_batch_instr)

# Export Report  
Calculations from Section 1 can only be integrated into the Master Table by functions from Section 2.

To revisit individual reports (or keep a record of analysis) without rerunning the notebook code:
- Save a .ipynb copy of this report to the main folder of your chosen experiment.  
- Save a html copy of this report to the main folder of your chosen experiment.  
- The easiest way to save a html report _without code cells_ is to use a browser plugin like Save Page WE.

<div style="font-size:14px; font-weight:bold; color:red">For all tables to be exported properly, ensure you have displayed the non-interactive Pandas table</div>
